<a href="https://colab.research.google.com/github/youdanzh/CIS700-projects/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import torch as tr
import matplotlib.pyplot as pt
from tqdm.notebook import trange
import numpy as np

In [12]:
examples = []
header = True

with open("data/00001", "r") as f:
  counter = 0
  clause = ""
  for line in f:
    counter += 1
    if line[0] == "T" and counter == 3:
      clause = line[2:]
    if line[0] in "+-":
      header = False
      label = 0 if line[0] == "-" else 1
    elif not header:
      clause_steps = line[2:] + " & " + clause
      #print(clause_steps)
      examples.append((clause_steps.strip(),label))
      
#for e,(example,label) in enumerate(examples[:10]):
 # print(label, example)
for i in range(10):
  print(examples[i])
len(examples)

('c==> c/\\ creal_ge creal_of_int f0 creal_of_num cNUMERAL c_0 c/\\ creal_ge creal_of_int f1 creal_of_num cNUMERAL c_0 c/\\ creal_ge creal_of_int f2 creal_of_num cNUMERAL c_0 c/\\ creal_ge creal_of_int f3 creal_of_num cNUMERAL c_0 c/\\ c/\\ creal_ge creal_add creal_mul creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_int f2 creal_add creal_of_int f3 creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_num cNUMERAL c_0 c= creal_of_int f1 creal_of_num cNUMERAL c_0 c/\\ c/\\ creal_ge creal_add creal_mul creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_int f2 creal_add creal_of_int f3 creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_num cNUMERAL c_0 c= creal_of_int f0 creal_of_num cNUMERAL c_0 creal_ge creal_add creal_mul creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_int f0 creal_add creal_of_int f1 creal_neg creal_of_num cNUMERAL cBIT1 c_0 creal_of_num cNUMERAL c_0 c/\\ c/\\ creal_ge creal_of_int f0 creal_of_num cNUMERAL c_0 c/\\ creal_ge creal_of_int f1 creal_of_num cNUMERAL c_0

540

In [13]:
validation = []
header = True

with open("00030", "r") as f:
  counter = 0
  clause = ""
  for line in f:
    counter += 1
    if line[0] == "T" and counter == 3:
      clause = line[2:]
    if line[0] in "+-":
      header = False
      label = 0 if line[0] == "-" else 1
    elif not header:
      clause_steps = line[2:] + " & " + clause
      #print(clause_steps)
      validation.append((clause_steps.strip(),label))
      
      


In [4]:
chrs = "".join([chr(c) for c in range(ord("a"), ord("z"))] + [chr(c) for c in range(ord("A"), ord("Z"))] + ["_"])
print(chrs)

def parse(line):
  tokens = []
  token = ""
  for c in line:
    if c in chrs: token += c
    else:
      tokens.append(token.lstrip())
      token = c
  tokens.append(token.lstrip())
  return tokens

print(parse(examples[0][0]))

abcdefghijklmnopqrstuvwxyABCDEFGHIJKLMNOPQRSTUVWXY_
['c', '=', '=', '>', 'c', '/', '\\', 'creal_ge', 'creal_of_int', 'f', '0', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '/', '\\', 'creal_ge', 'creal_of_int', 'f', '1', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '/', '\\', 'creal_ge', 'creal_of_int', 'f', '2', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '/', '\\', 'creal_ge', 'creal_of_int', 'f', '3', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '/', '\\', 'c', '/', '\\', 'creal_ge', 'creal_add', 'creal_mul', 'creal_neg', 'creal_of_num', 'cNUMERAL', 'cBIT', '1', 'c_', '0', 'creal_of_int', 'f', '2', 'creal_add', 'creal_of_int', 'f', '3', 'creal_neg', 'creal_of_num', 'cNUMERAL', 'cBIT', '1', 'c_', '0', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '=', 'creal_of_int', 'f', '1', 'creal_of_num', 'cNUMERAL', 'c_', '0', 'c', '/', '\\', 'c', '/', '\\', 'creal_ge', 'creal_add', 'creal_mul', 'creal_neg', 'creal_of_num', 'cNUMERAL', 'cBIT', '1', 'c_', '0', 'creal_of_int', 'f', '2', 'creal_add', 'c

In [5]:
all_tokens = set()
for example, label in examples:
  tokens = parse(example) 
  all_tokens |= set(tokens)

for example_2, label_2 in validation:
  tokens = parse(example_2) 
  all_tokens |= set(tokens)


all_tokens = list(all_tokens)
lookup = {token: t for (t, token) in enumerate(all_tokens)}

print(lookup)
for token in all_tokens: print(token)
print(len(all_tokens))


{'': 0, 'creal_le': 1, 'creal_div': 2, '0': 3, '<': 4, 'b': 5, '>': 6, 'creal_neg': 7, 'cwithin': 8, '+': 9, 'cT': 10, '3': 11, '*': 12, '~': 13, '/': 14, 'creal_of_int': 15, '1': 16, '=': 17, 'creal_gt': 18, 'cSUBSET': 19, 'f': 20, '4': 21, '-': 22, 'cat': 23, 'chas_complex_derivative': 24, 'creal_mul': 25, 'cSUC': 26, 'cint_add': 27, 'cint_lt': 28, 'c': 29, 'cF': 30, 'cint_of_num': 31, 'cNUMERAL': 32, '&': 33, '!': 34, 'ccomplex_mul': 35, 'creal_lt': 36, 'c_': 37, 'cBIT': 38, 'cint_le': 39, 'creal_of_num': 40, '\\': 41, 'creal_ge': 42, 'creal_add': 43, 'chas_derivative': 44, '2': 45}

creal_le
creal_div
0
<
b
>
creal_neg
cwithin
+
cT
3
*
~
/
creal_of_int
1
=
creal_gt
cSUBSET
f
4
-
cat
chas_complex_derivative
creal_mul
cSUC
cint_add
cint_lt
c
cF
cint_of_num
cNUMERAL
&
!
ccomplex_mul
creal_lt
c_
cBIT
cint_le
creal_of_num
\
creal_ge
creal_add
chas_derivative
2
46


In [6]:
max_len = max([len(parse(example)) for example,label in examples])
print("max_len:", max_len)
embeddings = tr.eye(len(all_tokens))

max_len: 651


In [7]:
"""
Transformer utilities
"""
def embed(seq, max_len, embeddings, offset=0):
    embedded = tr.zeros(max_len, embeddings.shape[1])
    cap = min(len(seq), max_len-offset)
    if cap > 0:
        embedded[offset:offset+cap] = tr.stack(tuple(embeddings[lookup[token]] for token in seq[:cap]))
    return embedded

def Attention(Q, K, V, masked=False):
    dk = Q.shape[1]
    logits = Q @ K.t() / dk**.5
    if masked:
        idx = tr.arange(Q.shape[0])
        logits[idx.unsqueeze(1) < idx] = -tr.inf
    return tr.softmax(logits, dim=1) @ V

class MultiHeadAttention(tr.nn.Module):
    def __init__(self, num_heads, d_model, masked=False, projections=""):
        super(MultiHeadAttention, self).__init__()
        dh = d_model // num_heads
        self.masked = masked
        self.num_heads = num_heads
        self.WQ, self.WK, self.WV = tuple(
            tr.nn.ModuleList([tr.nn.Linear(d_model, dh, bias=False) for i in range(num_heads)])
            if p in projections else [lambda x: x[:,:dh]]*num_heads
            for p in "QKV")
        self.WO = tr.nn.Linear(dh * num_heads, d_model, bias=False) if "O" in projections else lambda x: x
        self.ln = tr.nn.LayerNorm(d_model)
        self.projections = projections

    def forward(self, Q, K, V):
        heads = [
            Attention(self.WQ[i](Q), self.WK[i](K), self.WV[i](V), self.masked)
            for i in range(self.num_heads)]
        out = self.WO(tr.cat(heads, dim=1))
        out += Q # skip connection
        out = self.ln(out) # layer normalization
        return out

# requires d_model - d_embedding >= max_len
def one_hot_positional_encoder(max_len):
    def encode_position(inputs):
        I = tr.eye(max_len)
        return tr.cat((I, inputs), dim=1)
    return encode_position



In [8]:
class Net(tr.nn.Module):
  def __init__(self, num_blocks, num_heads):
    super(Net, self).__init__()
    d_model = max_len+embeddings.shape[1]
    self.encoder = one_hot_positional_encoder(max_len)
    self.blocks = tr.nn.ModuleList([
      MultiHeadAttention(num_heads, d_model, projections="QKVO")
      for _ in range(num_blocks)
    ])
    self.readout = tr.nn.Linear(d_model, 2)
  def forward(self, example):
    x = embed(parse(example), max_len, embeddings)
    x = self.encoder(x)
    for mha in self.blocks:
      x = mha(x, x, x)
    y = self.readout(x).mean(dim=0).unsqueeze(0)
    return y

net = Net(3, 4)
y = net(examples[0][0])
print(y)

tensor([[0.0137, 0.0868]], grad_fn=<UnsqueezeBackward0>)


In [10]:
validation = []
header = True

with open("data_valid/0017", "r") as f:
  counter = 0
  clause = ""
  for line in f:
    counter += 1
    if line[0] == "T" and counter == 3:
      clause = line[2:]
    if line[0] in "+-":
      header = False
      label = 0 if line[0] == "-" else 1
    elif not header:
      clause_steps = line[2:] + " & " + clause
      #print(clause_steps)
      validation.append((clause_steps.strip(),label))
      
print(validation)

[('c==> cbounded * * part cIMAGE f0 part cUNIV c= cbounded * * part cIMAGE f0 part cUNIV cT\n & c==> c/\\ cbilinear f0 c/\\ cmeasurable_on f1 part cUNIV c/\\ cbounded * * part cIMAGE f1 part cUNIV cabsolutely_integrable_on f2 part cUNIV cabsolutely_integrable_on / * * f0 * f1 cvector_sub f3 b0 * f2 b0 part cUNIV', 1), ('c==> ! ! ! c==> c/\\ cbilinear f0 c/\\ cabsolutely_integrable_on b0 part cUNIV c/\\ cmeasurable_on b1 part cUNIV cbounded * * part cIMAGE b1 part cUNIV cabsolutely_integrable_on / * * f0 * b1 b3 * b0 cvector_sub b2 b3 part cUNIV c= ! ! c==> c/\\ cbilinear f0 c/\\ cabsolutely_integrable_on b0 part cUNIV c/\\ cmeasurable_on f1 part cUNIV cbounded * * part cIMAGE f1 part cUNIV cabsolutely_integrable_on / * * f0 * f1 b2 * b0 cvector_sub b1 b2 part cUNIV cT\n & c==> c/\\ cbilinear f0 c/\\ cmeasurable_on f1 part cUNIV c/\\ cbounded * * part cIMAGE f1 part cUNIV cabsolutely_integrable_on f2 part cUNIV cabsolutely_integrable_on / * * f0 * f1 cvector_sub f3 b0 * f2 b0 part cUNIV

In [ ]:
net = Net(2, 4)
xc = tr.nn.CrossEntropyLoss()
if tr.cuda.is_available():
    net= net.cuda()

opt = tr.optim.Adam(net.parameters(), lr=0.0001)

num_iters = 200
verb_step = 20
train_loss = []
valid_accu =[]
valid_loss =[]
for i in trange(num_iters):

    example, label = random.choice(examples)
    if tr.cuda.is_available():
          example , label = example.cuda(), label.cuda()
    logits = net(example)
    loss = xc(logits, tr.tensor([label]))
    train_loss.append(loss.item())

    opt.zero_grad()
    loss.backward()
    opt.step()
    correct = []
    vloss = []
    
    
    if i % verb_step == 0 or i == num_iters-1:
       
        with tr.no_grad():
            for example_2, label_2 in validation:
               if tr.cuda.is_available():
                    example_2, label_2 = example_2.cuda(), label_2.cuda()
               logits = net(example_2)
               v_loss = xc(logits,tr.tensor([label_2]))  
               pred = logits.argmax()
               correct.append(np.absolute(label_2-pred))
               vloss.append(v_loss.item())
        valid_accu.append(1-np.mean(correct))
        valid_loss.append(np.mean(vloss))
        print(f'Epoch:{i} loss:{loss.item()}\tval_loss: {np.mean(vloss)}\tval_acc: {1-np.mean(correct)}  ')
        
              

#pt.plot(prediction)
pt.plot(train_loss)
pt.xlabel("Iteration")
pt.ylabel("Loss")



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch:0 loss:0.7275561094284058	val_loss: 0.6917630136013031	val_acc: 0.5  
Epoch:20 loss:0.38888177275657654	val_loss: 0.6813520044088364	val_acc: 0.5  
Epoch:40 loss:0.6238937377929688	val_loss: 0.6948464810848236	val_acc: 0.5  


In [ ]:
'''Youdan's Approch'''

class Net(tr.nn.Module):
  def __init__(self, num_blocks, num_heads):
    super(Net, self).__init__()
    d_model = max_len+embeddings.shape[1]
    self.conv = tr.nn.Conv1d(max_len,d_model, kernel_size=1)    
    self.encoder = one_hot_positional_encoder(max_len)
    self.blocks = tr.nn.ModuleList([
      MultiHeadAttention(num_heads, d_model, projections="QKVO")
      for _ in range(num_blocks)
    ])
    self.readout =tr.nn.Linear(d_model, 2)
    
  def forward(self, example):
    x = embed(parse(example), max_len, embeddings)
    x = self.encoder(x)
    x = self.conv(x)
    for mha in self.blocks:
      x = mha(x, x, x)
    y = self.readout(x).mean(dim=0).unsqueeze(0)
    return y

net = Net(2, 4)

In [ ]:
net = Net(4, 8)
xc = tr.nn.CrossEntropyLoss()
if tr.cuda.is_available():
    net= net.cuda()

opt = tr.optim.Adam(net.parameters(), lr=0.001)

num_iters = 1000
verb_step = 20
train_loss = []
valid_accu =[]
valid_loss =[]
for i in trange(num_iters):

    example, label = random.choice(examples)
    if tr.cuda.is_available():
          example , label = example.cuda(), label.cuda()
    logits = net(example)
    loss = xc(logits, tr.tensor([label]))
    train_loss.append(loss.item())

    opt.zero_grad()
    loss.backward()
    opt.step()
    correct = []
    vloss = []
    
    
    if i % verb_step == 0 or i == num_iters-1:
       
        with tr.no_grad():
            for example_2, label_2 in validation:
               if tr.cuda.is_available():
                    example_2, label_2 = example_2.cuda(), label_2.cuda()
               logits = net(example_2)
               v_loss = xc(logits,tr.tensor([label_2]))  
               pred = logits.argmax()
               correct.append(np.absolute(label_2-pred))
               vloss.append(v_loss.item())
        valid_accu.append(1-np.mean(correct))
        valid_loss.append(np.mean(vloss))
        print(f'loss:{loss.item()} \t\tval_loss: {np.mean(vloss)}\t\t val_acc: {1-np.mean(correct)}  ')
        
              

#pt.plot(prediction)
pt.plot(train_loss)
pt.xlabel("Iteration")
pt.ylabel("Loss")


  0%|          | 0/1000 [00:00<?, ?it/s]

loss:0.7791317701339722 		val_loss: 0.712788537144661		 val_acc: 0.5  
loss:0.1580270677804947 		val_loss: 1.128480363637209		 val_acc: 0.5  
loss:1.3389838933944702 		val_loss: 0.7905789464712143		 val_acc: 0.5  
loss:0.6077137589454651 		val_loss: 0.7152821123600006		 val_acc: 0.5  
loss:0.3863331079483032 		val_loss: 0.8968489728868008		 val_acc: 0.5  
loss:0.5246638059616089 		val_loss: 0.720523789525032		 val_acc: 0.5  
loss:0.47818723320961 		val_loss: 0.7455583736300468		 val_acc: 0.5  
loss:0.868634045124054 		val_loss: 0.7007477879524231		 val_acc: 0.5  
loss:0.463378369808197 		val_loss: 0.7618705555796623		 val_acc: 0.5  
loss:0.9469958543777466 		val_loss: 0.7167620286345482		 val_acc: 0.5  
loss:1.082108497619629 		val_loss: 0.7157258987426758		 val_acc: 0.5  
loss:1.0535938739776611 		val_loss: 0.7335361465811729		 val_acc: 0.5  
loss:0.7050405144691467 		val_loss: 0.6931810081005096		 val_acc: 0.5  
loss:0.7858015298843384 		val_loss: 0.6933953911066055		 val_acc: 0.5  
